In [1]:
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

from scipy.io import loadmat
from scipy.io import savemat
import pandas as pd
import numpy as np
import transformations  as tr
import random as rand
from sklearn.model_selection import train_test_split
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5486289639353733052
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1460528742
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10973933851610603356
physical_device_desc: "device: 0, name: GeForce GTX 760, pci bus id: 0000:01:00.0, compute capability: 3.0"
]


In [3]:
def generatePairs(frames):
    framePairs = [] #vector to hold image pairs

    for x in range(len(frames)): #x represents the first frame in the pair
        for y in range(len(frames)-3-x): #y represents second frame in the pair
            pair = [(x+1),(x+y+4),np.stack((frames[x], frames[x+y+3]), axis=-1)] #generate pair
            framePairs.append(pair)
    return framePairs

In [4]:
def translatePair(pair,steps,patient):

    frame1 = pair[0]-1 #get first frame position from pair object
    frame2 = pair[1]-1 #get second frame position from pair object
    JAdd = pair[2].shape[0] #Get size of original frame to keep translation size consistent
    IAdd = pair[2].shape[1] #Get size of original frame to keep translation size consistent

    baseCut = 0.1 #use same base cut as is used in original frame generation
    step = baseCut/steps #set step size to be as large as possible for given number of steps
    pairs = [] #vector to hold newly generated pairs

    #adjust how the frame is cropped from the original data to simulate translation
    for x in range(-steps,steps+1): 
        for y in range(-steps,steps+1):
            lowerI = int((cutLeft+x*step)*256)
            upperI = lowerI+IAdd
            lowerJ = int((cutTop+y*step)*2592)
            upperJ = lowerJ+JAdd

            first = rawData[patient][1][lowerJ:upperJ,lowerI:upperI,frame1]
            second = rawData[patient][1][lowerJ:upperJ,lowerI:upperI,frame2]

            pairs.append(np.stack((first,second), axis=-1)) #append translated pair to pairs vector
    return pairs

In [5]:
def normalize(min,max,array):
    test[test < min] = min
    test[test > max] = max
    array = tr.unit_vector(array)
    return array

In [6]:
#Directory containing the RF scan data
dataFolder = 'C:/Users/Lucas/OneDrive - The University of Western Ontario/Documents/GitHub/SE4450-project-code/Preprocessing/'

#Vector to hold all training data sets
rawData = []

#Loading in the matlab file information
rawMatFile = loadmat('P39-W2-S4')
#Contains just the RF data pulled from the .mat file
P39W2S4 = rawMatFile['rf1']
rawData.append(['P39W2S4',P39W2S4])

#repeat above steps for other RF files
rawMatFile = loadmat('P39-W4-S6')
P39W4S6 = rawMatFile['rf1']
rawData.append(['P39W4S6',P39W4S6])
# rawMatFile = loadmat('P82-W0-S2')
# P82W0S2 = rawMatFile['rf1']
# rawData.append(['P82W0S2',P82W0S2])
rawMatFile = loadmat('P87-W0-S3')
P87W0S3 = rawMatFile['rf1']
rawData.append(['P87W0S3',P87W0S3])
rawMatFile = loadmat('P87-W2-S4')
P87W2S4 = rawMatFile['rf1']
rawData.append(['P87W2S4',P87W2S4])
rawMatFile = loadmat('P90-W0-S4')
P90W0S4 = rawMatFile['rf1']
rawData.append(['P90W0S4',P90W0S4])
rawMatFile = loadmat('P94-W1-S3')
P94W1S3 = rawMatFile['rf1']
rawData.append(['P94W1S3',P94W1S3])

In [7]:
#leave a slice of each edge of the scan from being black
cutTop = 0.1
cutLeft = 0.1
cutBottom = 0.1
cutRight = 0.1

lowerI = int(cutLeft*256)
upperI = int((1-cutRight)*256)
lowerJ = int(cutTop*2592)
upperJ = int((1-cutBottom)*2592)

allFrames = [] #vector to hold individual cropped frames

for y in range(len(rawData)):
    frames = []
    for x in range(rawData[y][1].shape[2]): #range is how many frames there are in the set
        frames.append(rawData[y][1][lowerJ:upperJ,lowerI:upperI,x]) #Take inner subset of frame to allow for translation
    allFrames.append([rawData[y][0],frames])

pairs = []
for x in range(len(allFrames)):
    pairs.append([allFrames[x][0],generatePairs(allFrames[x][1])]) #add generated pairs to list
print(pairs[5][1][0][2].shape)

#PAIRS VARIABLE GUIDE
#First value = Holds which patient set is being referred to
#Second value = Holds either patient set name [0] or generated pairs for set
#Third value = Holds all pairs for patient set
#Fourth value = holds specific pairs for patient set
#pairs[0][1][0][2].shape <- references the shape of the third pair of frames for a certain patients data

(1805, 205, 2)


In [8]:
#Create Vector of stacked pairs with frame number and patient ID for easy labeling
data = []
for x in range(len(pairs)):
    for y in range(len(pairs[x][1])):
        data.append([pairs[x][0],pairs[x][1][y][0],pairs[x][1][y][1],pairs[x][1][y][2]])

print(len(data))
#for x in range(200):
#    print(data[x][0],data[x][1],data[x][2],data[x][3].shape)

998


In [9]:
#Read in label data
DFP39W2S4 = pd.read_excel('Labels.xlsx', sheet_name='P39-W2-S4', engine='openpyxl')
DFP39W4S6 = pd.read_excel('Labels.xlsx', sheet_name='P39-W4-S6', engine='openpyxl')
#DFP82W0S2 = pd.read_excel('Labels.xlsx', sheet_name='P82-W0-S2', engine='openpyxl')
DFP87W0S3 = pd.read_excel('Labels.xlsx', sheet_name='P87-W0-S3', engine='openpyxl')
DFP87W2S4 = pd.read_excel('Labels.xlsx', sheet_name='P87-W2-S4', engine='openpyxl')
DFP90W0S4 = pd.read_excel('Labels.xlsx', sheet_name='P90-W0-S4', engine='openpyxl')
DFP94W1S3 = pd.read_excel('Labels.xlsx', sheet_name='P94-W1-S3', engine='openpyxl')

In [10]:
# + DFP82W0S2['Label axial'].tolist() wrong shape?
#create labels for training data
labels = DFP39W2S4['Label axial'].tolist() + DFP39W4S6['Label axial'].tolist()  + DFP87W0S3['Label axial'].tolist() + DFP87W2S4['Label axial'].tolist() + DFP90W0S4['Label axial'].tolist() + DFP94W1S3['Label axial'].tolist()
print(len(labels))

998


In [11]:
labeledData = []
for x in range(len(labels)):
    labeledData.append([data[x][3],labels[x]])

In [12]:
goodData = []
badData = []
for x in range(len(labeledData)):
    #Setting meh labels to 0
    if labeledData[x][1] == 0.5:
        labeledData[x][1] = 0

    if labeledData[x][1] > 0:
        goodData.append(labeledData[x])
    else:
        badData.append(labeledData[x])
print(len(goodData))
print(len(badData))
badData = rand.sample(badData,len(goodData))
print(len(goodData))
print(len(badData))
balancedData = goodData+badData
print(type(balancedData))

159
839
159
159
<class 'list'>


In [13]:
def train_preprocessing(data, label):
    data = tf.expand_dims(data, axis=3)
    return data, label


def validation_preprocessing(data, label):
    data = tf.expand_dims(data, axis=3)
    return data, label

In [14]:
# Split data in the ratio 80-20 for training and validation.
x_train, x_test, y_train, y_test = train_test_split([x[0] for x in balancedData], [x[1] for x in balancedData], test_size=0.20, random_state=42)

print(y_train)

[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0, 1.0, 0, 0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0, 1.0, 1, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1, 1.0, 0.0, 1, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0, 1, 0, 0, 1.0, 0.0, 1.0, 0, 1.0, 0.0, 1.0, 0.0, 0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0, 0.0, 1.0, 0.0, 1.0, 0.0, 0, 0, 0.0, 0.0, 1.0, 0, 1.0, 1.0, 0.0, 0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0, 1.0, 0.0, 1.0, 1.0, 0, 0.0, 0, 1.0, 1.0, 0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 0, 0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0, 1.0, 0.0, 1.0, 1

In [15]:
for x in range(len(x_train)):
    x_train[x] = tf.convert_to_tensor(x_train[x], np.float32)

for x in range(len(x_test)):
    x_test[x] = tf.convert_to_tensor(x_test[x], np.float32)

In [16]:
# Define data loaders.
train_loader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
validation_loader = tf.data.Dataset.from_tensor_slices((x_test, y_test))

batch_size = 2
# Augment the on the fly during training.
train_dataset = (
    train_loader.shuffle(len(x_train))
    .map(train_preprocessing)
    .batch(batch_size)
    .prefetch(2)
)
# Only rescale.
validation_dataset = (
    validation_loader.shuffle(len(x_test))
    .map(validation_preprocessing)
    .batch(batch_size)
    .prefetch(2)
)

In [17]:
def get_model(width=1805, height=205, depth=2):
    """Build a 3D convolutional neural network model."""

    inputs = keras.Input((width, height, depth, 1))

    x = layers.Conv3D(filters=64, kernel_size=(3,3,2), activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=(2,2,1))(x)
    x = layers.BatchNormalization()(x)

    # x = layers.Conv3D(filters=64, kernel_size=(3,3,2), activation="relu")(x)
    # x = layers.MaxPool3D(pool_size=(2,2,1))(x)
    # x = layers.BatchNormalization()(x)

    # x = layers.Conv3D(filters=128, kernel_size=(3,3,2), activation="relu")(x)
    # x = layers.MaxPool3D(pool_size=(2,2,1))(x)
    # x = layers.BatchNormalization()(x)

    # x = layers.Conv3D(filters=256, kernel_size=(3,3,2), activation="relu")(x)
    # x = layers.MaxPool3D(pool_size=(2,2,1))(x)
    # x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=512, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(units=1, activation="sigmoid")(x)

    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn")
    return model


# Build model.
model = get_model(width=1805, height=205, depth=2)
model.summary()


Model: "3dcnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1805, 205, 2, 1)] 0         
_________________________________________________________________
conv3d (Conv3D)              (None, 1803, 203, 1, 64)  1216      
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 901, 101, 1, 64)   0         
_________________________________________________________________
batch_normalization (BatchNo (None, 901, 101, 1, 64)   256       
_________________________________________________________________
global_average_pooling3d (Gl (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 512)               33280     
_________________________________________________________________
dropout (Dropout)            (None, 512)               0     

In [3]:
# Compile model.
initial_learning_rate = 0.0001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)
model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
    metrics=["acc"],
)

# Define callbacks.
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    "3d_image_classification.h5", save_best_only=True
)
early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_acc", patience=15)

# Train the model, doing validation at the end of each epoch
epochs = 50
model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=epochs,
    shuffle=True,
    verbose=2,
    callbacks=[checkpoint_cb, early_stopping_cb],
)


NameError: name 'keras' is not defined